This file aims at creating sarcastic and non sarcastic labels based on polarity of the text. 

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import nltk

from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from mlxtend.preprocessing import DenseTransformer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
from google.colab import drive

drive.mount('/content/gDrive/', force_remount=True)

Mounted at /content/gDrive/


In [ ]:
df = pd.read_csv('/content/gDrive/My Drive/Colab/Dissertation/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1',header=None)
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
df=df[[5,0]]
df.columns=['tweets','sentiment']
df.head()

,tweets,sentiment
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


#preprocessing the tweets

In [ ]:
# Reading contractions.csv and storing it as a dict.
contractions = pd.read_csv('/content/gDrive/My Drive/Colab/Dissertation/contractions.csv', index_col='Contraction')
contractions.index = contractions.index.str.lower()
contractions.Meaning = contractions.Meaning.str.lower()
contractions_dict = contractions.to_dict()['Meaning']

# Defining regex patterns.
urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|(www\.)[^ ]*)"
userPattern       = '@[^\s]+'
hashtagPattern    = '#[^\s]+'
alphaPattern      = "[^a-z0-9<>]"
sequencePattern   = r"(.)\1\1+"
seqReplacePattern = r"\1\1"

# Defining regex for emojis
smileemoji        = r"[8:=;]['`\-]?[)d]+"
sademoji          = r"[8:=;]['`\-]?\(+"
neutralemoji      = r"[8:=;]['`\-]?[\/|l*]"
lolemoji          = r"[8:=;]['`\-]?p+"

def preprocess_apply(tweet):

    tweet = tweet.lower()

    # Replace all URls with '<url>'
    tweet = re.sub(urlPattern,'<url>',tweet)
    # Replace @USERNAME to '<user>'.
    tweet = re.sub(userPattern,'<user>', tweet)
    
    # Replace 3 or more consecutive letters by 2 letter.
    tweet = re.sub(sequencePattern, seqReplacePattern, tweet)

    # Replace all emojis.
    tweet = re.sub(r'<3', '<heart>', tweet)
    tweet = re.sub(smileemoji, '<smile>', tweet)
    tweet = re.sub(sademoji, '<sadface>', tweet)
    tweet = re.sub(neutralemoji, '<neutralface>', tweet)
    tweet = re.sub(lolemoji, '<lolface>', tweet)

    for contraction, replacement in contractions_dict.items():
        tweet = tweet.replace(contraction, replacement)

    # Remove non-alphanumeric and symbols
    tweet = re.sub(alphaPattern, ' ', tweet)

    # Adding space on either side of '/' to seperate words (After replacing URLS).
    tweet = re.sub(r'/', ' / ', tweet)
    return tweet

In [ ]:
%%time
df['processed_tweet'] = df.tweets.apply(preprocess_apply)

CPU times: user 1min 35s, sys: 500 ms, total: 1min 35s
Wall time: 1min 41s


In [ ]:
df.head()

,tweets,sentiment,processed_tweet
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,<user> <url> aww thatis a bummer you shou...
1,is upset that he can't update his Facebook by ...,0,is upset that he cannot update his facebook by...
2,@Kenichan I dived many times for the ball. Man...,0,<user> i dived many times for the ball manage...
3,my whole body feels itchy and like its on fire,0,my whole body feels itchy and like its on fire
4,"@nationwideclass no, it's not behaving at all....",0,<user> no it is not behaving at all i am mad...


In [ ]:
count=0
for row in df.itertuples():
    print("Tweet:", row[2])
    print("Processed:", row[3])
    count+=1
    if count>10:
        break

Tweet: 0
Processed: <user> <url>   aww  thatis a bummer   you shoulda got david carr of third day to do it  <smile>
Tweet: 0
Processed: is upset that he cannot update his facebook by texting it   and might cry as a result  school today also  blah 
Tweet: 0
Processed: <user> i dived many times for the ball  managed to save 50   the rest go out of bounds
Tweet: 0
Processed: my whole body feels itchy and like its on fire 
Tweet: 0
Processed: <user> no  it is not behaving at all  i am mad  why am i here  because i cannot see you all over there  
Tweet: 0
Processed: <user> not the whole crew 
Tweet: 0
Processed: need a hug 
Tweet: 0
Processed: <user> hey  long time no see  yes   rains a bit  only a bit  lol   i am fine thanks   how is you  
Tweet: 0
Processed: <user> nope they did not have it 
Tweet: 0
Processed: <user> que me muera   
Tweet: 0
Processed: spring break in plain city   it is snowing 


In [ ]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:

def tokenize(column):
    tokens = nltk.word_tokenize(column)
    return [w for w in tokens if w.isalpha()]

def remove_stopwords(tokenized_column):

    stops = set(stopwords.words("english"))
    return [word for word in tokenized_column if not word in stops]
#stemming
def apply_stemming(tokenized_column):
    stemmer = PorterStemmer() 
    return [stemmer.stem(word) for word in tokenized_column]

def rejoin_words(tokenized_column):
    return ( " ".join(tokenized_column))

In [ ]:
df['tokenize'] = df.apply(lambda x: tokenize(x['processed_tweet']), axis=1)

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df['stopwords_removed'] = df.apply(lambda x: remove_stopwords(x['tokenize']), axis=1)

In [ ]:
df['porter_stemmed'] = df.apply(lambda x: apply_stemming(x['stopwords_removed']), axis=1)

In [ ]:
df['rejoined'] = df.apply(lambda x: rejoin_words(x['porter_stemmed']), axis=1)

In [ ]:
df['rejoined'].value_counts(dropna=False)

user                                                                7299
user thank                                                          3653
user get follow day use url add everyon train pay vip               1484
user welcom                                                          880
work                                                                 805
                                                                    ... 
ahh someon told dirti mouth shall resist                               1
slept hour went home sick work ah joy offic contamin matter time       1
tht soo crzi love six flag                                             1
energi pill suck felt like go die                                      1
happi charitytuesday user user user                                    1
Name: rejoined, Length: 1476770, dtype: int64

In [ ]:
countVectorizer = CountVectorizer(analyzer=rejoin_words) 
bow = countVectorizer.fit_transform(df['processed_tweet'])

In [ ]:
# Initialize a CountVectorizer object: count_vectorizer
count_vectorizer = CountVectorizer( encoding='latin-1', ngram_range=(1, 2), stop_words='english')

# Transform the training data using only the 'text' column values: count_train 
count_train = count_vectorizer.fit_transform(df['processed_tweet'])

# Transform the test data using only the 'text' column values: count_test 
count_test = count_vectorizer.transform(df['processed_tweet'])

# Print the first 10 features of the count_vectorizer
print(count_vectorizer.get_feature_names()[:20])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['00', '00 00', '00 001', '00 00pm', '00 00th', '00 01', '00 01am', '00 01s', '00 02', '00 03', '00 05', '00 05am', '00 06', '00 09', '00 10', '00 100', '00 100th', '00 11', '00 12', '00 13']


In [ ]:
bow = np.array(bow.todense())

In [ ]:
df.head()

,tweets,sentiment,processed_tweet,tokenize,stopwords_removed,porter_stemmed,rejoined
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,<user> <url> aww thatis a bummer you shou...,"[user, url, aww, thatis, a, bummer, you, shoul...","[user, url, aww, thatis, bummer, shoulda, got,...","[user, url, aww, thati, bummer, shoulda, got, ...",user url aww thati bummer shoulda got david ca...
1,is upset that he can't update his Facebook by ...,0,is upset that he cannot update his facebook by...,"[is, upset, that, he, can, not, update, his, f...","[upset, update, facebook, texting, might, cry,...","[upset, updat, facebook, text, might, cri, res...",upset updat facebook text might cri result sch...
2,@Kenichan I dived many times for the ball. Man...,0,<user> i dived many times for the ball manage...,"[user, i, dived, many, times, for, the, ball, ...","[user, dived, many, times, ball, managed, save...","[user, dive, mani, time, ball, manag, save, re...",user dive mani time ball manag save rest go bound
3,my whole body feels itchy and like its on fire,0,my whole body feels itchy and like its on fire,"[my, whole, body, feels, itchy, and, like, its...","[whole, body, feels, itchy, like, fire]","[whole, bodi, feel, itchi, like, fire]",whole bodi feel itchi like fire
4,"@nationwideclass no, it's not behaving at all....",0,<user> no it is not behaving at all i am mad...,"[user, no, it, is, not, behaving, at, all, i, ...","[user, behaving, mad, see]","[user, behav, mad, see]",user behav mad see


In [ ]:
X = bow
y = df['sentiment']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [ ]:
model = MultinomialNB().fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1 score:', f1_score(y_test, y_pred, average="macro"))
print('ROC AUC:', roc_auc_score(y_test, y_pred))

Accuracy: 0.5958916666666667
F1 score: 0.5955292590851592
ROC AUC: 0.5958916666666667


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.59      0.63      0.61    240000
           4       0.60      0.57      0.58    240000

    accuracy                           0.60    480000
   macro avg       0.60      0.60      0.60    480000
weighted avg       0.60      0.60      0.60    480000



In [ ]:
results = pd.DataFrame(data={'predicted': y_pred, 'actual': y_test})
predictions = results.join(df)

In [ ]:
def is_correct(predicted, actual):
    if predicted == actual:
        return True
    else:
        return False

predictions['correct'] = predictions.apply(lambda x: is_correct(x.predicted, x.actual), axis=1)
predictions = predictions[['tweets','predicted','actual','correct']]

In [ ]:
predictions[predictions['correct']==False].sample(10)

,tweets,predicted,actual,correct
1051355,I DIDNT LIE!! ALL I DID WAS CON YOU IN 2 VOTIN...,0,4,False
869172,cleaning &amp;&amp; lovin' it! Txt it,0,4,False
600884,@officialTila I LOVE your new profile pic! Whe...,4,0,False
755691,Dang it! @elscooby and I missed our 1st conce...,4,0,False
1229256,@ClaireHammond need to know who is doing schoo...,0,4,False
611420,@xJessicaaax Sorry no can do your little Nep...,4,0,False
854612,@Jenlagos Hehe I am using Twitter! I enjoyed b...,0,4,False
988379,is soooo high......................,0,4,False
1429391,Tennis and pool time took up the whole day. N...,0,4,False
324093,@mizrik good 2 hear. Still waiting on our iho...,4,0,False


here actual column in the table shows the sarcastic and non sarcastic sentences based on polarity, 0 = negative, 4 = positive.

In [ ]:
predictions.head()

,tweets,predicted,actual,correct
390952,"I'm mad, I erased all the pics in my camera by...",0,0,True
1215041,@dancedrug ok that's fine. I hope you had a g...,4,4,True
1411594,Chilling out with his second vodka and coke an...,0,4,False
1071872,Walmart: hair dye and ice cream.,0,4,False
821882,rainy evening...nap turned into 2 1/2 hours,0,4,False


In [ ]:
predictions.tail()

,tweets,predicted,actual,correct
1192413,laying out,4,4,True
416315,:sight: finally I approved it all... fuih... d...,0,0,True
373859,@mikeziegler I didn't have anyone 2 go with. Y...,4,0,False
882202,Woody jumper http://twitpic.com/4h02e,4,4,True
1540941,"finals today are over, early release all week",4,4,True


In [ ]:
predictions[predictions['correct']==True].sample(10)

,tweets,predicted,actual,correct
504749,Got home from anatomy not to long ago and now ...,0,0,True
604490,what a morning rain sucks!!,0,0,True
914574,@kaitgoesrawr26 just got them- about to read &...,4,4,True
714809,bak from showtyme BBQ! It was so much fun exc...,0,0,True
286763,havin lunch w/o my sweetie... wishin he waz ...,0,0,True
977305,@ddlovato hm. are you okay?,4,4,True
520652,@tweetdeck problem with v0.25.2b: window geome...,0,0,True
1324208,@Fixedr6 We can avoid being swept up in it. Li...,4,4,True
1132591,Just saw &quot;UP!&quot; it was sooo cute aw...,4,4,True
927702,@eeeethannnn you tweeted &quot;apple&quot; so ...,4,4,True
